In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
#some required values
batch_size = 32
number_of_classes = 10
epochs = 100 # for testing; use epochs = 100 for training ~30 secs/epoch on CPU
weight_decay = 1e-4
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'
number_of_images = 5
learning_rate = 0.0001
decay = 1e-6

In [ ]:
labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
# load the data  and inspect its shape
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


In [ ]:
def show_images(images):
    plt.figure(1)
    image_index = 0
    for i in range(0,number_of_images):
        for j in range(0,number_of_images):
            plt.subplot2grid((number_of_images, number_of_images),(i,j))
            plt.imshow(Image.fromarray(images[image_index]))
            image_index +=1
            plt.gca().axes.get_yaxis().set_visible(False)
            plt.gca().axes.get_xaxis().set_visible(False)   
    plt.show()
    
show_images(x_test[:number_of_images*number_of_images])

In [ ]:
# Normalise the data and convert
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255


In [ ]:
# Convert labels to one-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, number_of_classes) # or use tf.one_hot()
y_test = tf.keras.utils.to_categorical(y_test, number_of_classes)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(number_of_classes, activation='softmax'))


In [ ]:
# initialise the optimiser
opt = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])


In [ ]:
print('Using data augmentation in real-time.')
    # Preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range 0 to 10 degrees
        
        width_shift_range=0.1,# randomly shift images horizontally (fraction of total width)
        
        height_shift_range=0.1,# randomly shift images vertically (fraction of total height)
    
        horizontal_flip=True,  # randomly flip images
    
        validation_split=0.1)

In [ ]:
# datagen.fit(x_train) 
# (this is only needed if any of the feature-wise normalizations i.e. 
# std, mean, and principal components ZCA whitening are set to True.)

# set things up to halt training if the accuracy  has stopped increasing
# could also monitor = 'val' or monitor = 
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,  batch_size=batch_size), epochs=epochs, callbacks=[callback])
# Save model and weights

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Model saved at: %s ' % model_path)

In [ ]:
model1 = tf.keras.models.load_model(model_path)
model1.summary()

In [ ]:
# Evaluate  our trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:

#reload the data since it has been mangled
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
show_images(x_test[:number_of_images*number_of_images])
x_test = x_test.astype('float32')/255

In [ ]:
indices = tf.argmax(input=model1.predict(x_test[:number_of_images*number_of_images]),axis=1)
i = 0
print('Learned  True')
print('=====================')
for index in indices:
    print(labels[index], '\t', labels[y_test[i][0]])
    i+=1